# Using BERT to generate emojis 

BERT is a transformer based model used mainly for classification tasks. In this notebook, we will use Bert Base model to fine-tune an emoji-producing classification system. In essence, on the top of the base model, we will add a linear layer. This is used to output a tensor of the following shape: batch size * number of labels. 

We will in essence, use the prediction probablities from BERT to compare to a an emotion to emoji word embedding space using Euclidian Distance and Cosine Similarity. The top 3 emojis that are closest to the our vectorized predictions will be outputted. 


In [ ]:
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.1 MB/s eta 0:00:00


In [ ]:
!pip install datasets
#!pip install -q transformers datasets
#!pip install -q --upgrade transformers
#!pip install -q --upgrade datasets
!pip install -q accelerate

!pip uninstall -y transformers accelerate
!pip install transformers accelerate

!pip install -q --upgrade accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.7 MB/s eta 0:00:00
Found existing installation: transformers 4.30.0
Uninstalling transformers-4.30.0:
  Successfully uninstalled transformers-4.30.0
Found existing installation: accelerate 0.20.3
Uninstalling accelerate-0.20.3:
  Successfully uninstalled accelerate-0.20.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.30.0-py3-none-any.whl (7.2 MB)
  Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing the data

For our study, we will utilize a Semantic Evaluation task dataset sourced from 2018. Specifically, we will focus on the English version of these tasks. The dataset consists of two primary components: Tweets obtained from the Twitter platform and manually assigned emotions to the corresponding text. These emotions are selected by a group of crowdworkers through a meticulous manual process.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("sem_eval_2018_task_1", "subtask5.english")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

Dataset sem_eval_2018_task_1 downloaded and prepared to /root/.cache/huggingface/datasets/sem_eval_2018_task_1/subtask5.english/1.1.0/a7c0de8b805f1988b118882fb289ccfbbeb9085c7820b6f046b5887e234af182. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

The dataset includes training, validation, and testing. 

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 6838
    })
    test: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 3259
    })
    validation: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 886
    })
})

What does one example look like?

In [ ]:
example = dataset['train'][0]
example

{'ID': '2017-En-21441',
 'Tweet': "“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry",
 'anger': False,
 'anticipation': True,
 'disgust': False,
 'fear': False,
 'joy': False,
 'love': False,
 'optimism': True,
 'pessimism': False,
 'sadness': False,
 'surprise': False,
 'trust': True}

We will use a mapping system to identify the labels given and id, and also the id given the label. 

In [ ]:
labels = [label for label in dataset['train'].features.keys() if label not in ['ID', 'Tweet']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['anger',
 'anticipation',
 'disgust',
 'fear',
 'joy',
 'love',
 'optimism',
 'pessimism',
 'sadness',
 'surprise',
 'trust']

## Preprocessing

To work with models like BERT, text needs to be tokenized using a tokenizer. We use the AutoTokenizer API, which selects the appropriate tokenizer based on the checkpoint.

For multi-label text classification, labels should be formatted as a matrix of shape (batch_size, num_labels). It's important to represent the labels as floats instead of integers to avoid issues with PyTorch's BCEWithLogitsLoss.

BERT requires input in vector format, so we preprocess the data by tokenizing sentences into word lists. Labels are reshaped to match the expected dimensions of the model.

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["Tweet"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/6838 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

Map:   0%|          | 0/886 [00:00<?, ? examples/s]

In [ ]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
tokenizer.decode(example['input_ids'])

"[CLS] “ worry is a down payment on a problem you may never have '. joyce meyer. # motivation # leadership # worry [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

In [ ]:
example['labels']

[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['anticipation', 'optimism', 'trust']

Changing our tensors to fit PyTorch

In [ ]:
encoded_dataset.set_format("torch")

## Define model

In the code snippet provided, we define a model that combines a pre-trained base (specifically, the weights from bert-base-uncased) with a classification head initialized randomly using a linear layer. It's crucial to fine-tune both the classification head and the pre-trained base on a labeled dataset to achieve optimal performance.

Please note that a warning is displayed to draw attention to this requirement.

For the purpose of multi-label classification, we set the problem_type as "multi_label_classification". This ensures the usage of the appropriate loss function, namely BCEWithLogitsLoss. Additionally, we configure the output layer to have the same number of neurons as the total number of labels in the dataset. Lastly, we establish the necessary mappings between label IDs and their corresponding labels.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

## Training

We trained using HuggingFace's Training API. Here are the two objects we used: Training Arguments and the Trainer object. 

In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We will output a dictionary of metrics we produce

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result


We will proceed with training the model using the training set and evaluate its performance on the validation set. As we observed signs of overfitting with the BERT model when using a high number of epochs, we have made the decision to limit the training to only 5 epochs.

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.221400,0.322991,0.700401,0.794918,0.274266
2,0.196600,0.322523,0.708892,0.804415,0.278781


In [ ]:
# import torch
# torch.save(trainer, '/content/drive')

# saved_model = trainer.load('/content/drive')

## Evaluating

We will evalute trainer on the evaluation and testing set. 

In [ ]:
trainer.evaluate()

In [ ]:
test_result = trainer.evaluate(eval_dataset=encoded_dataset["test"])
print(test_result)

## Inserting our text

We will tokenize our text and output the emotion vector. 

In [ ]:
text = "I am so shocked that you came home!"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The output logits from the model have a shape of (batch_size, num_labels). Since we are passing only a single sentence through the model, the batch size is set to 1. The logits tensor contains the unnormalized scores corresponding to each individual label.

In [ ]:
logits = outputs.logits
logits.shape

## Final Steps
We will use a sigmoid function to essentially turn every predicted label between 0 and 1. The closer to 1, the higher the chance that the emotion label is related to the text that is given. 

We then chose a threshold of 1/2 to turn every probability into a 1 or 0 if the threshold is not met. 

### Emotions from the Twitter dataset
anger, anticipation, disgust, fear, joy, love, optimism, pessimism, sadness, surprise, trust

### Emotion from EmoTag dataset
anger, anticipation, disgust, fear, joy, sadness, surprise, trust

### What we will do to level out emotions
The Twitter dataset has 3 emotions in addition to the EmoTag dataset: love, optimism, pessimism. Thus, we will shrink the prediction probability vector of emotions of the Twitter dataset to fit the amount of emotions in the emotions in the EmoTag dataset. We will combine the emotions as follows:

1. Love -> Add half to the Joy and Trust. 
2. Optimisim -> Add to Anticipation
3. Pessimism -> Add half to Disgust and Fear. 

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)

print(probs)

# Convert tensor to a regular Python list
tensor_list = probs.tolist()

print(tensor_list)

# Index for where emotion is
love= 5
optimism = 6
pessimism = 7
joy = 4
trust = 10
anticipation = 1
fear = 3
disgust = 2

print(predictions)

# Add
probs[joy] += probs[love] / 2
probs[trust] += probs[love] / 2
probs[anticipation] += probs[optimism]
probs[disgust] += probs[pessimism] / 2
probs[fear] += probs[pessimism] / 2

# Make values 0 at extra indexes
probs[love] = 0 
probs[pessimism] = 0
probs[optimism] = 0


# Convert tensor to a regular Python list
tensor_list = probs.tolist()

print(tensor_list)

predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)
print(predictions)

# Changing type into regular list
predictions = predictions.tolist()

# Get rid of the specific columns
positions_to_remove = [love, optimism, pessimism]

for index in sorted(positions_to_remove, reverse=True):
    del tensor_list[index]

print(tensor_list)



In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.spatial.distance import jaccard,hamming

# Load the CSV file
url = 'https://raw.githubusercontent.com/abushoeb/EmoTag/37d27757d90965d9a9b8a4ff712fe6533e9bb544/data/EmoTag1200-scores-details.csv?plain=1'

# Define the desired columns to use, removed std deviation and other metadata
columns_to_use = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust', 'emoji']

# Read the DataFrame with the desired columns
df = pd.read_csv(url, usecols=columns_to_use)

# Define the input vector
input = [tensor_list]

# Calculate cosine similarity for each emoji
for emotion in input:
  matched_emojis = []
  matched_emojis_two = []

  for _, row in df.iterrows():
      vector = row.iloc[:-1].values

      # compute cosine similarity
      similarity = cosine_similarity(np.array(emotion).reshape(1, -1), vector.reshape(1, -1))
      matched_emojis.append((row['emoji'], similarity[0][0]))

      # compute euclidean_distance
      euclidean_distance = np.linalg.norm(np.array(emotion) - np.array(vector))
      matched_emojis_two.append((row['emoji'], euclidean_distance))

  # Sort the matched emojis by similarity (in descending order)
  matched_emojis.sort(reverse=True, key=lambda x: x[1])
  matched_emojis_two.sort( key=lambda x: x[1])

  # Print the top 3 matched emojis
  print('Text: ', text)
  print('Top Emotions for Text', predicted_labels)
  print('Cosine Similarity:',[matched_emoji[0] for matched_emoji in matched_emojis[:3]])
  print('Euclidean Distance:',[matched_emoji[0] for matched_emoji in matched_emojis_two[:3]])
  print()